a) Read up on the OpenINTEL project on the website and in the respective publication2
to briefly answer the
following questions: What kinds of measurements are OpenINTEL performing? What are some challenges of
the measurements? Which resource records are measured for each domain? For which labels/subdomains of
the domains are the queries performed? (2 points)

OpenINTEL performs both forward and reverse DNS measurements.
It performs active measurements (sends queries), so they have a consistent and reliable state of the global DNS.

There are some challenges:

A large-scale active measurement: daily collect
data for the largest top level domain. ".com" requires at least 1.85B queries per day.

Large Storage: more than 240GB
of results need to be stored per day for .com alone

The program Robustness: it should run continuously, not crash

The impacts of the global DNS infrastructure: this scale of active measurement should not impose an unacceptable
burden, and so on

SOA
NS
A
AAAA
MX
TXT
DNSKEY
DS
NSEC3
CAA
CDS
CDNSKEY, these records are measured every second-level domain in a TLD, also for www label, non-existent domain name to record authenticated denial-of-existence data
, A and AAAA records for these records in a separate infrastructure measurement, TLSA records for ports 25, 465 and 587 in a separate infrastructure measurement.

Every second-level domain in a TLD and 'www' labels are the queries performed.

b) Write a program that downloads DNS measurements for Alexa Top 1M and Cisco Umbrella from OpenINTEL3
for a specified date. Extend your program so that it untars the files and opens the .avro files within, and inspect
the file schema and format. Make your script extract all responses that correspond to A (IPv4 addresses), AAAA
(IPv6 addresses), and CNAME resource records for each queried www. subdomain, and store the results (along
with the measurement date) in a SQL database; distinguish between the Alexa and Umbrella list accordingly by
creating separate tables. Finally, extend your script to repeat this process for measurements of a whole month
(specified by the user). (2 points)

solution b):

The program is "question_b.py". To start it you should provide a multiprocessing environment.
Note: it is recommended to start the program with max buffer size 10000

An example is: **"question_b.py alexa 2020 10 --buffer 10000"**
```
usage: question_b.py [-h] [--day [day]] [--db [db_name]] [--cache [cache_dir]]
                     [--buffer [buffer_max]]
                     source year month

question_b

positional arguments:
  source                the source of the data, can be Alexa or Umbrella
  year                  the year to query, example: 10
  month                 the month to query, example: 2

optional arguments:
  -h, --help            show this help message and exit
  --day [day]           the day to query, if given, only one day will be
                        handled
  --db [db_name]        the db name (name only), example: "example.db"
  --cache [cache_dir]   the cache folder, example: data
  --buffer [buffer_max]
                        the max buffer size: the size of avro entries that are
                        buffered in the memory

```

In order to lead to memory overflow, there is a "buffer_size" parameter that
can limit the number of the records buffered inside the "dao". If the "buffer_size" is set to 0,
which means the buffer size is not limited, then all of the records of "avro-file" will be loaded
at once and save to the sql, so the speed will be faster than using a buffer size limitation.

The extracting of data from "avro-file" is implemented with multiprocessing, each "avro-file" will be
processed by one thread (process), so the speed can be fast.

The downloading can similarly also be implemented with multiprocessing,
but downloading too many files and extracting all "avro-files" may cost too much storage. In this reason,
the software will clean all of the tar- and avro- files immediately after finished the extracting.

The program provides 2 DAO(Data Access Object) files: "dao" and "pandao".
"dao" uses the pure python "sqlite3" lib, and "pandao" uses the "pandas" lib.

In [ ]:
# the program can also be run as a function call
from question_b import handle_one_month

# note: slow! about 10 min!
handle_one_month(2020, 10, db_name="example.db", buffer_max=10000)
# will print: "used time: 878.292760, total: 3682335"


c) Read up on the Route Views project to briefly answer the following questions: What kinds of data are Route
Views collecting? What is a Routing Information Base (RIB) in this context? How are the collected RIBs different
from BGP UPDATE data? (2 points)

solution c):

collecting real-time BGP information.

RIB is a package or table like this:
```
TIME: 10/15/20 12:00:00
TYPE: TABLE_DUMP_V2/IPV4_UNICAST
PREFIX: 0.0.0.0/0
SEQUENCE: 0
FROM: 80.249.210.150 AS199524
ORIGINATED: 08/14/20 13:35:07
ORIGIN: IGP
ASPATH: 199524 174
NEXT_HOP: 80.249.210.150
```
It contains the AS path information (ASPATH), which shows
the real path of a package.

BGP UPDATE data only shows the accessibility of each BGP routers,
but the RIB shows the real path of a package. BGP routers may use some strategies,
they may prefer to send the packages to a "far" router based on the policies.
So the RIB can show the real routing among AS.

d) Write a program that adds columns for the AS numbers of the Autonomous Systems (ASes), which announce
the IPv4 and IPv6 addresses of the resolved domains, to each table of your database. To map IP addresses to
AS numbers (e.g., with pyasn
), use BGP data collected from the Amsterdam Internet Exchange (AMS-IX),
which you can download from the Route Views archive
. For simplicity, it is sufficient to take one RIB file (e.g.,
for the 15th of the month at 12:00 PM) for all IP-ASN-mappings over the whole month. Why do we choose data
from AMS-IX rather than from other collectors? (2 points)

solution d):

Why do we choose data from AMS-IX rather than from other collectors?
Because of the timezone.

The program is "question_d.py".  To start it you should provide a multiprocessing environment and an existed database.
Note: it is recommended to start the program with chunksize 1000

An example is: **"question_d.py alexa 2020 10 --chunksize 1000"**

```
usage: question_d.py [-h] [--db [db_name]] [--cache [cache_dir]] [--chunksize [chunksize]] [--process [process]] source year month

question_d

positional arguments:
  source                the source of the data, can be Alexa or Umbrella
  year                  the year to query, example: 10
  month                 the month to query, example: 2

optional arguments:
  -h, --help            show this help message and exit
  --db [db_name]        the db name (name only), example: "example.db"
  --cache [cache_dir]   the cache folder, example: data
  --chunksize [chunksize]
                        the max chunksize to read from sql table every time
  --process [process]   the number of process to read and update the sql table

```

The program will download the AMS-IX RIB archives with the giving year and month
and convert it to IPASN databases, use it for looking up the as.

In [ ]:
# the program can also be run as a function call
from question_d import Asn

# default 50 processes
asn = Asn("example.db", 'alexa',year=2020, month=10, chunksize=1000, process_number=50)
asn.flush_ases()
# used time: 643.814716



e) Read up on different mechanisms with which CDNs (and similarly cloud providers) achieve client redirection7
.
Download the regexes.csv and asns.csv files from Moodle (note that the provided files will not exhaustively
and correctly identify all cloud/CDN providers8
, however, the accuraccy will be sufficient for this assignment).
Since the AS numbers of the providers in the list were looked up with both RIPEstat9 and PeeringDB10, some
provider-ASN-mapping might be duplicate—you can simply drop such duplicates and only keep distinct tuples.
Briefly describe how and why the information provided by these files can be used to identify cloud/CDN providers.
Write a program that adds a column to each table of your database for the identified website host based on the
two files and following criteria:
1. If a CNAME for a domain exists, apply the regular expressions to potentially identify the website host.
2. If no website host was found via CNAME, identify the website host via the AS numbers for IPv4 and IPv6.
3. If neither CNAME nor ASN identified the website host, put -1 as the website host.


solution e):

The program is "question_e.py".  To start it you should provide a multiprocessing environment and an existed database.
Note: it is recommended to start the program with chunksize 1000

An example is: **"question_d.py alexa --chunksize 1000"**

The options of "question_e.py" are similar with d).

In [ ]:
# the program can also be run as a function call
from question_e import Host

# default 50 processes
h = Host("example.db", 'alexa', chunksize=1000, process_number=50)
h.flush_host()
# used time: 841.023135

f) How many unique websites (absolute and relative count) does each website host (incl. -1, i.e., non-cloud and
non-CDN hosts) serve over the whole month? Provide a CSV file (columns: host, num_uniq_websites)
and sort the hosts in descending order. Does your observation support or contradict the perception described in
the beginning of the assignment? Do you see significant differences between the Alexa and Umbrella data?
(2 points)

solution f):

The program is "question_f.py".
For the csv, I use a subset of the data(data of the day: 20201015).
But the program can use for the whole month.


The csv files are 'Umbrella_out.csv' and 'Alexa_out.csv'

The observe is the same with my perception: the most websites are small webs without CDN and the most CDNs are from the big cloud server companies.

We can see that Cloudflare, Amazon are the most popular CDNs w.r.t. the Umbrella source,
and most websites are without CDNs, and it is different from Alexa.

g) Read up on different Internet toplists11 to briefly answer the following questions: Which different toplists
exist? What is the difference between them? How are the lists generated? What can be said about their
stability? (2 points)


Alexa: http traffic, Umbrella: DNS /IPs, Majestic:backlinks, Quantcast: visit number

h) Write a program that downloads and unpacks the respective Alexa and Umbrella toplists for the whole month
from a toplist archive12. Consider that the naming scheme of the files in the archive changed in May 2018.
Load the daily toplists into a dataframe or table each. Write a program to enrich these daily toplists with the
website host column from your database by matching the domain names. Note that you may need to add
the www. subdomain to the domains for a successful join. For each Alexa and Umbrella, save the resulting
dataframes/tables with toplist rank, domain name, IPv4/IPv6, website host, and date as a new table in your SQL
database. (2 points)

solution h):

The program is "question_h.py".
